In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Github/openssa

/content/drive/MyDrive/Github/openssa


In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
!pip install -q google-auth-oauthlib==0.7.0
!pip install -q tensorflow==2.15.1
!pip install -q langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.2 MB/s eta 0:00:00


## Load dataset

In [13]:
import pandas as pd
import json
from pandas import json_normalize
import zipfile

import re

In [14]:
TRAIN_PATH = "examples/elementary-math/data/dataset/train.zip"
TEST_PATH = "examples/elementary-math/data/dataset/test.zip"

In [15]:
def read_from_zip(path_zip, filename):
  json_data = None
  with zipfile.ZipFile(path_zip, "r") as z:
        with z.open(filename) as f:
          data = f.read()
          json_data = json.loads(data)
  return json_normalize(json_data['data'])

In [16]:
df = read_from_zip(TRAIN_PATH, "math_train.json")
df

,id,question,choices,explanation,answer
0,1,"Một người bán hàng bỏ ra 80,000 đồng tiền vốn ...","[A. 80,000 : 6, B. 80,000 x 6, C. 80,000 : (6 ...","Theo đề bài, số tiền lỗ bằng 6% của 80 000 đồn...","D. (80,000 x 6) : 100"
1,10,8 dm2 24 cm2 = ……… dm2. Số thích hợp điền v...,"[A. 824, B. 82,4, C. 8,24, D. 0,824]","Ta có 24 cm2 = 0,24 dm2\n Vậy 8 dm2 24 cm2 = ...","C. 8,24"
2,100,"10% của 11,5m2 là:","[A. 10,15dm2, B. 1,5m2, C. 15,5m2, D. 1,15m2]","10% của 11,5m2 là: 11,5 ${\times}$ 10 : 100 = ...","D. 1,15m2"
3,1004,"Một tam giác có đáy bằng 15 cm và gấp 2,5 l...","[A. 90 cm2, B. 37,5 cm2, C. 45 cm2, D. 18,75 cm2]","Chiều cao của tam giác đó là: 15 : 2,5 = 6 (c...",C. 45 cm2
4,1005,Một tấm thiệp hình chữ nhật có chiều rộ...,"[A. 162 cm2, B. 81 cm2, C. 99 cm2, D. 63 cm2]",Chiều dài hình chữ nhật là: 9 ${\times}$ 2 ...,A. 162 cm2
...,...,...,...,...,...
1195,991,"Một ô tô đi được 60 km với vận tốc 60 km/giờ, ...","[A. 1,5 giờ, B. 2 giờ, C. 3 giờ, D. 4 giờ]","vì đoạn đường thứ nhất ô tô đi hết 1 giờ, đoạn...",C. 3 giờ
1196,993,"Một ô tô đi từ A đến B với vận tốc 38,5k...","[A. 2 giờ 1 phút, B. 2 giờ 6 phút, C. 2 giờ...","Thời gian ô tô đã đi là: 100,1 : 38,5 = 2,6 (...",C. 2 giờ 36 phút
1197,994,Một ô tô đi từ Hà Nội lúc 11 giờ trưa và đến V...,"[A. 4 giờ 30 phút, B. 6 giờ 30 phút, C. 6 giờ ...",5 giờ 30 phút chiều = 17 giờ 30 phút\n Thời gi...,D. 6 giờ
1198,999,Một sân vận động hình chữ nhật có chiều ...,"[A.380 m, B.285 m, C.670 m, D.760 m]",Chiều dài sân vận động là: 95 ${\times}$ 3 ...,D.760 m


## Translate from Vietnamese to English

In [17]:
def document_formating(row):
  return f"Câu hỏi: {row['question']}. Các phương án: {row['choices']}. Giải thích: {row['explanation']}. Đáp án: {row['answer']}"

In [18]:
df['document'] = df.apply(lambda x: document_formating(x), axis=1)

In [19]:
questions_vi = df['question'].to_list()
documents_vi = df['document'].to_list()

In [20]:
def split_into_batchs(lst, batch_size):
    return [lst[i:i+batch_size] for i in range(0, len(lst), batch_size)]

questions_vi_batchs = split_into_batchs(questions_vi, 100)
documents_vi_batchs = split_into_batchs(documents_vi, 100)

In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

questions_en = []
for batch in questions_vi_batchs:
  input_ids = tokenizer(batch, return_tensors="pt", padding=True).input_ids.to(device)
  outputs = model.generate(input_ids, max_length=512)
  questions_en = questions_en + tokenizer.batch_decode(outputs, skip_special_tokens=True)

documents_en = []
for batch in documents_vi_batchs:
  input_ids = tokenizer(batch, return_tensors="pt", padding=True).input_ids.to(device)
  outputs = model.generate(input_ids, max_length=512)
  documents_en = documents_en + tokenizer.batch_decode(outputs, skip_special_tokens=True)

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.49M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [17]:
df['question_en'] = questions_en
df['documents_en'] = documents_en
df.to_csv('examples/elementary-math/data/rag/train_df_en.csv', index=False)

## Processing

In [16]:
from transformers import AutoTokenizer
from langchain.text_splitter import RecursiveCharacterTextSplitter


llm = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(llm)

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(tokenizer, chunk_size=256, chunk_overlap=2, separators=["\n\n", "\n", ". "])


section_text = "Hello. This is some text to split. With a few "\
                "uncharacteristic words to chunk, expecting 2 chunks."

texts = text_splitter.split_text(section_text)
print(texts)

['Hello. This is some text to split. With a few uncharacteristic words to chunk, expecting 2 chunks.']


In [17]:
for text in texts:
  print(text)

Hello. This is some text to split. With a few uncharacteristic words to chunk, expecting 2 chunks.


## Chunks is different from tokens

In [29]:
section_text = "Hello. This is some text to split. With a few "\
                "uncharacteristic words to chunk, expecting 2 chunks."
encoded_text = tokenizer(section_text)
tokens = tokenizer.convert_ids_to_tokens(encoded_text['input_ids'])
print(tokens)

['▁Hello', '.', '▁This', '▁is', '▁some', '▁text', '▁to', '▁split', '.', '▁With', '▁', 'a', '▁few', '▁un', 'character', 'istic', '▁words', '▁to', '▁chunk', ',', '▁expecting', '▁2', '▁chunk', 's', '.', '</s>']


## Using RAG for multiple choices question

### Embedding Vectors

In [18]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [19]:
!pip install -q sentence-transformers

In [20]:
from sentence_transformers import SentenceTransformer, util

embedding_model_path = "BAAI/bge-small-en-v1.5"

embedding_model = SentenceTransformer(embedding_model_path)

#### Embedding the question

In [21]:
df_test = read_from_zip(TEST_PATH, "math_test.json")
df_test['question'][0]

'Một cửa hàng đã bán 30% số hàng hiện có và thu được 15 000 000 đồng. Hỏi nếu bán hết hàng thì cửa hàng thu được bao nhiêu tiền?'

In [22]:
question = df_test['question'][0]

q_embeddings = embedding_model.encode(question)

In [23]:
q_embeddings.shape

(384,)

q_embeddings is a single vector and has the length of 384 (other models may use longer vectors for more accurate relations, ex: https://huggingface.co/spaces/mteb/leaderboard)

In [35]:
q_embeddings[:10]

array([-0.03703814, -0.07685137,  0.04402602, -0.05147359, -0.01734495,
       -0.05069024,  0.00479159,  0.03354499, -0.01825728,  0.01302371],
      dtype=float32)

## Embedding the question-explaination documents

### Embed document chunks

In [38]:
documents_en = pd.read_csv('examples/elementary-math/data/rag/document_en.csv')['0'].to_list()

In [40]:
BATCH_SIZE = 100  # you can submit up to 2048 embedding inputs per request

embeddings = []
for batch_start in range(0, len(documents_en), BATCH_SIZE):
    batch_end = batch_start + BATCH_SIZE
    batch = documents_en[batch_start:batch_end]
    print(f"Batch {batch_start} to {batch_end-1}")
    response = embedding_model.encode(batch)
    batch_embeddings = [e for e in response]
    embeddings.extend(batch_embeddings)

df = pd.DataFrame({"text": documents_en, "embedding": embeddings})

Batch 0 to 99
Batch 100 to 199
Batch 200 to 299
Batch 300 to 399
Batch 400 to 499
Batch 500 to 599
Batch 600 to 699
Batch 700 to 799
Batch 800 to 899
Batch 900 to 999
Batch 1000 to 1099
Batch 1100 to 1199


In [41]:
df

,text,embedding
0,"en: Answer: D. (80,000 x 6) : 100","[-0.029183704, 0.058640204, 0.040616862, -0.05..."
1,en: Question: 8 dm2 24 cm2 =......... dm2. The...,"[-0.09648136, -0.028235849, -0.030202378, 0.01..."
2,en: Question: 10% of 11.5m2 is:. Methods: ['A....,"[-0.056904368, -0.029418064, -0.04563393, 0.00..."
3,"en: The solutions: ['A. 90 cm2', 'B. 37.5 cm2'...","[-0.105948314, -0.019610714, -0.023260837, 0.0..."
4,en: Question: The length of each letter-shaped...,"[-0.037589293, 0.02249079, -0.0093185, 0.00440..."
...,...,...
1195,en: Question: A car travels 60 km at 60 km/h a...,"[-0.063412674, -0.0012685272, 0.019707305, 0.0..."
1196,en: Question: The cars traveled from A to B at...,"[-0.037900794, 0.06481222, -0.006223498, 0.032..."
1197,en: Question: A car leaves Hanoi at 11: 00 p. ...,"[-0.055856988, 0.03199247, -0.018042082, -0.00..."
1198,"en: The range of the pitch is: ['A. 380 m', 'B...","[-0.03644165, 0.002973021, -0.030741686, -0.03..."


In [42]:
SAVE_PATH = "examples/elementary-math/data/rag/embedded_documents_en.csv"
df.to_csv(SAVE_PATH, index=False)

In [43]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a, b):
    return dot(a, b)/(norm(a)*norm(b))

In [44]:
sim_scores = [cos_sim(q_embeddings, e) for e in df["embedding"].values]

In [45]:
df["sim_score"] = sim_scores

In [46]:
df

,text,embedding,sim_score
0,"en: Answer: D. (80,000 x 6) : 100","[-0.029183704, 0.058640204, 0.040616862, -0.05...",0.592060
1,en: Question: 8 dm2 24 cm2 =......... dm2. The...,"[-0.09648136, -0.028235849, -0.030202378, 0.01...",0.523718
2,en: Question: 10% of 11.5m2 is:. Methods: ['A....,"[-0.056904368, -0.029418064, -0.04563393, 0.00...",0.581655
3,"en: The solutions: ['A. 90 cm2', 'B. 37.5 cm2'...","[-0.105948314, -0.019610714, -0.023260837, 0.0...",0.540900
4,en: Question: The length of each letter-shaped...,"[-0.037589293, 0.02249079, -0.0093185, 0.00440...",0.486902
...,...,...,...
1195,en: Question: A car travels 60 km at 60 km/h a...,"[-0.063412674, -0.0012685272, 0.019707305, 0.0...",0.509229
1196,en: Question: The cars traveled from A to B at...,"[-0.037900794, 0.06481222, -0.006223498, 0.032...",0.489198
1197,en: Question: A car leaves Hanoi at 11: 00 p. ...,"[-0.055856988, 0.03199247, -0.018042082, -0.00...",0.623799
1198,"en: The range of the pitch is: ['A. 380 m', 'B...","[-0.03644165, 0.002973021, -0.030741686, -0.03...",0.473823


In [47]:
df.sort_values(by=["sim_score"], ascending=False).head(10)

,text,embedding,sim_score
57,en: Question: Phu Quy is an island in Phan Thi...,"[-0.031672865, 0.0172046, -0.036668546, -0.016...",0.677158
275,en: The answer: C. 15 916,"[-0.030368613, 0.014574511, 0.010705572, -0.06...",0.671122
782,en: Answer: A.898,"[-0.03937657, -0.019976612, -0.007974199, -0.0...",0.667207
1091,en: Question: A bank spends $15 000 000 in cap...,"[-0.004131848, -0.021487685, -0.038455762, -0....",0.656105
1189,en: Answer: A. 0.75m,"[-0.032545574, 0.014360184, -0.019717796, -0.0...",0.655092
658,en: Question: Mai's family has four adults and...,"[-0.0049529737, 0.00086068606, -0.04091457, -0...",0.645220
467,"en: Answer: A. 54 900, 55 000, 55 100 ',' B. 5...","[-0.04969977, -0.013395858, -0.041126337, -0.0...",0.641414
360,en: Question: Uncle An sent us a deposit of $2...,"[-0.020572992, 0.032011297, -0.03542361, -0.04...",0.641163
965,"en: Question: A person deposits $1,000,000 in ...","[-0.023342036, -0.004451796, -0.061640903, 0.0...",0.636835
814,en: Question: Hoa bought four broken copies an...,"[-0.07599921, 0.0047969464, -0.02256086, -0.00...",0.635587


In [48]:
df_sorted_sim_score = df.sort_values(by=["sim_score"], ascending=False).reset_index()
df_sorted_sim_score['text'][0]

"en: Question: Phu Quy is an island in Phan Thiet province. There are 30 971 people living on the island. If we round the number of people living on the island to the thousands, we can say that the number of people living on Phu Quy is about:. Methods: ['A. 30 000 people', 'B. 40 000 people', 'C. 31 000 people', 'D. 29 000 people']. Interpretation: To round the number 30 971 to the thousands we consider the hundreds digit. We have 9 > 5, so we round up. We get the number 31 000.."

## Re-ranking

In [49]:
SAVE_PATH = "examples/elementary-math/data/rag/embedded_documents_en.csv"
df = pd.read_csv(SAVE_PATH)

In [ ]:
rerank_model_path = "BAAI/bge-reranker-base"

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

rerank_tokenizer = AutoTokenizer.from_pretrained(rerank_model_path)
rerank_model = AutoModelForSequenceClassification.from_pretrained(rerank_model_path)
rerank_model.eval()

def calculate_rerank_scores(pairs):
    with torch.no_grad():
        inputs = rerank_tokenizer(pairs, padding=True,
                                  truncation=True,
                                  return_tensors='pt', max_length=512)
        scores = rerank_model(**inputs, return_dict=True).logits.view(-1, ).float()
        return scores

pairs = [(question, chunk) for chunk in df["text"].values]
rerank_scores = calculate_rerank_scores(pairs)
df["rerank_score"] = rerank_scores

## Building the context

In [ ]:
top_chunks = df.sort_values(by=["sim_score"], ascending=False).head(10)["text"].values
context_arr = []
for t in top_chunks:
    context_arr.append(t)

In [ ]:
context = "".join(context_arr)

context

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

llm_answer_path = "HuggingFaceH4/zephyr-7b-beta"
torch_device = "cuda"
tokenizer = AutoTokenizer.from_pretrained(llm_answer_path)
llm_answer = AutoModelForCausalLM.from_pretrained(llm_answer_path,
             device_map=torch_device,
             torch_dtype=torch.float16)

# assuming here that "context" contains the pre-built context
query = "answer the following question, "\
    "based on your knowledge and the provided context. "\
"Keep the answer concise.\n\nquestion:" + question + "\n\ncontext:"+ context

input_ids = tokenizer.encode(query+"\n\nANSWER:",
   return_tensors='pt', return_attention_mask=False).to(torch_device)
greedy_output = llm_answer.generate(input_ids,
                                 max_new_tokens=1024, do_sample=True)
answer = tokenizer.decode(greedy_output[0], skip_special_tokens=True)
print(answer[len(query):])